In [ ]:
#!pip install Pillow
#!pip install scipy==1.1.0

In [ ]:
! pip install SimpleITK

In [ ]:
import SimpleITK as sitk

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns 

import os
import glob

##import SimpleITK as sitk

from PIL import Image

from imageio import imread

import tensorflow as tf

%matplotlib inline
from IPython.display import clear_output
pd.options.mode.chained_assignment = None

In [ ]:
annotations = pd.read_csv('../input/luna16/annotations.csv')
candidates = pd.read_csv('../input/luna16/candidates.csv')

In [ ]:
annotations.head(5)

In [ ]:
candidates.head(5)

In [ ]:
print(len(candidates[candidates['class'] == 1]))
print(len(candidates[candidates['class'] == 0]))


In [ ]:
import multiprocessing
num_cores = multiprocessing.cpu_count()
print(num_cores)


In [ ]:
class CTScan(object):
    def __init__(self, filename = None, coords = None):
        self.filename = filename
        self.coords = coords
        self.ds = None
        self.image = None

In [ ]:
    def reset_coords(self, coords):
        self.coords = coords

    def read_mhd_image(self):
        path = glob.glob('../input/luna16/seg-lungs-LUNA16/seg-lungs-LUNA16/'+ self.filename + '.mhd')
        self.ds = sitk.ReadImage(path[0])
        self.image = sitk.GetArrayFromImage(self.ds)

    def get_resolution(self):
        return self.ds.GetSpacing()

    def get_origin(self):
        return self.ds.GetOrigin()

    def get_ds(self):
        return self.ds

    def get_voxel_coords(self):
        origin = self.get_origin()
        resolution = self.get_resolution()
        voxel_coords = [np.absolute(self.coords[j]-origin[j])/resolution[j] \
            for j in range(len(self.coords))]
        return tuple(voxel_coords)
    
    def get_image(self):
        return self.image
    
    def get_subimage(self, width):
        self.read_mhd_image()
        x, y, z = self.get_voxel_coords()
        subImage = self.image[z, y-width/2:y+width/2, x-width/2:x+width/2]
        return subImage   
    def normalizePlanes(self, npzarray):
        maxHU = 400.
        minHU = -1000.
        npzarray = (npzarray - minHU) / (maxHU - minHU)
        npzarray[npzarray>1] = 1.
        npzarray[npzarray<0] = 0.
        return npzarray
    
    def save_image(self, filename, width):
        image = self.get_subimage(width)
        image = self.normalizePlanes(image)
        Image.fromarray(image*255).convert('L').save(filename)

In [ ]:
positives = candidates[candidates['class']==1].index  
negatives = candidates[candidates['class']==0].index

In [ ]:
scan = CTScan(np.asarray(candidates.iloc[negatives[600]])[0], \
              np.asarray(candidates.iloc[negatives[600]])[1:-1])
scan.read_mhd_image()
x, y, z = scan.get_voxel_coords()
image = scan.get_image()
dx, dy, dz = scan.get_resolution()
x0, y0, z0 = scan.get_origin()

In [ ]:
filename = '1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793540579077826395208'
coords = (70.19, -140.93, 877.68)#[877.68, -140.93, 70.19]
scan = CTScan(filename, coords)
scan.read_mhd_image()
x, y, z = scan.get_voxel_coords()
image = scan.get_image()
dx, dy, dz = scan.get_resolution()
x0, y0, z0 = scan.get_origin()

In [ ]:
"../input/luna16/subset*/subset*/1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860"

In [ ]:
positives

In [ ]:
np.random.seed(42)
negIndexes = np.random.choice(negatives, len(positives)*5, replace = False)

In [ ]:
candidatesDf = candidates.iloc[list(positives)+list(negIndexes)]


In [ ]:
from sklearn.model_selection import train_test_split
X = candidatesDf.iloc[:,:-1]
y = candidatesDf.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.10, random_state = 42)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.20, random_state = 42)

In [ ]:
X_test.shape

In [ ]:
X_train.to_pickle('traindata')
X_test.to_pickle('testdata')
X_val.to_pickle('valdata')

In [ ]:
X_train.shape

In [ ]:
def normalizePlanes(npzarray):
    maxHU = 400.
    minHU = -1000.
    npzarray = (npzarray - minHU) / (maxHU - minHU)
    npzarray[npzarray>1] = 1.
    npzarray[npzarray<0] = 0.
    return npzarray


In [ ]:
print('number of positive cases are ' + str(y_train.sum()))
print('total set size is ' + str(len(y_train)))
print('percentage of positive cases are ' + str(y_train.sum()*1.0/len(y_train)))

In [ ]:
########################## Duplicate the positive class images ########################
tempDf = X_train[y_train == 1]
tempDf = tempDf.set_index(X_train[y_train == 1].index + 1000000)
X_train_new = X_train.append(tempDf)
tempDf = tempDf.set_index(X_train[y_train == 1].index + 2000000)
X_train_new = X_train_new.append(tempDf)

ytemp = y_train.reindex(X_train[y_train == 1].index + 1000000)
ytemp.loc[:] = 1
y_train_new = y_train.append(ytemp)
ytemp = y_train.reindex(X_train[y_train == 1].index + 2000000)
ytemp.loc[:] = 1
y_train_new = y_train_new.append(ytemp)

print(len(X_train_new), len(y_train_new))

In [ ]:
print('number of positive cases are ' + str(int(y_train_new.sum())))
print('total set size is ' + str(len(y_train_new)))
print('percentage of positive cases are ' + str(y_train_new.sum()*1.0/len(y_train_new)))

In [ ]:
X_train_new.head(5)

In [ ]:
y_train_new.head(5)

In [ ]:
X_train_new.index

In [ ]:
import scipy.misc
import cv2

In [ ]:
#from scipy.misc import imresize
from PIL import ImageEnhance

class PreProcessing(object):
    def __init__(self, image = None):
        self.image = image
    
    def subtract_mean(self):
        self.image = (self.image/255.0 - 0.25)*255
        return self.image
    
    def downsample_data(self):
        self.image = cv2.resize(self.image, (40,40), interpolation = cv2.INTER_AREA)
        return self.image

    def upsample_data(self):
        self.image = cv2.resize(self.image, (224, 224), interpolation = cv2.INTER_CUBIC)
        return self.image

In [ ]:
from imageio import imread
dirName = '../input/luna-image-data/data-20211016T064224Z-001/data/train/'
plt.figure(figsize = (10,10))
inp = imread(dirName + 'image_'+ str(1049439) + '.jpg')
print ("Original shape of input image: ", inp.shape)
plt.subplot(221)
plt.imshow(inp, cmap='gray')
plt.grid(False)

inp = PreProcessing(inp).upsample_data()
Pp = PreProcessing(inp)

inp2 = Pp.subtract_mean()
plt.subplot(222)
plt.imshow(inp2, cmap='gray')
plt.grid(False)

inp3 = ImageEnhance.Contrast(Image.fromarray(inp))
contrast = 1.5
inp3 = inp3.enhance(contrast)
plt.subplot(223)
plt.imshow(inp3, cmap='gray')
plt.grid(False)

inp4 = Pp.downsample_data()
plt.subplot(224)
plt.imshow(inp4,cmap='gray')
plt.grid(False)

In [ ]:
inp.shape

In [ ]:
inp.dtype

In [ ]:
y_train_new.values.astype(int)

In [ ]:
dirName

In [ ]:
train_filenames =\
X_train_new.index.to_series().apply(lambda x:\
                                    '../input/luna-image-data/data-20211016T064224Z-001/data/train/image_'+str(x)+'.jpg')
train_filenames.values.astype(str)

In [ ]:
train_filenames.values.astype(str)

In [ ]:
dataset_file = 'traindatalabels.txt'
filenames = train_filenames.values.astype(str)
train_filenames =X_train_new.index.to_series().apply(lambda x:filenames)
labels = y_train_new.values.astype(int)
traindata = np.zeros(filenames.size,\
                     dtype=[('var1', 'S36'), ('var2', int)])
traindata['var1'] = filenames
traindata['var2'] = labels

np.savetxt(dataset_file, traindata, fmt="%10s %d")

In [ ]:
traindata

In [ ]:
import h5py
h5f = h5py.File('../input/luna-image-data/data-20211016T064224Z-001/data/traindataset.h5', 'r')
X_train_images = h5f['X']
Y_train_labels = h5f['Y']

h5f2 = h5py.File('../input/luna-image-data/data-20211016T064224Z-001/data/valdataset.h5', 'r')
X_val_images = h5f2['X']
Y_val_labels = h5f2['Y']

In [ ]:
X_train_images

In [ ]:
Y_train_labels

In [ ]:
X_train_processing = np.array(X_train_images)
X_val_processing = np.array(X_val_images)

print(X_train_processing.shape)
print(X_val_processing.shape)

print(X_train_processing.dtype)
print(X_val_processing.dtype)

In [ ]:
img_test = X_train_processing[1000]
img_test_downsampled = PreProcessing(img_test).downsample_data()
img_test_upsampled = PreProcessing(img_test).upsample_data()

print(Y_train_labels[1000])

In [ ]:
plt.subplot(121)
plt.imshow(img_test,cmap='gray')
plt.subplot(122)
plt.imshow(img_test_downsampled,cmap='gray')

In [ ]:
plt.subplot(121)
plt.imshow(img_test,cmap='gray')
plt.subplot(122)
plt.imshow(img_test_upsampled,cmap='gray')

In [ ]:
img_test_subtracted_mean = PreProcessing(img_test_upsampled).subtract_mean()
plt.subplot(121)
plt.imshow(img_test_upsampled,cmap='gray')
plt.subplot(122)
plt.imshow(img_test_subtracted_mean,cmap='gray')

In [ ]:
import cv2
brisk = cv2.BRISK_create(50)
img_BRISK_1 = img_test_upsampled.copy()
rgb = cv2.cvtColor(img_BRISK_1, cv2.COLOR_GRAY2RGB)
img_BRISK_2 = rgb*1000
img_BRISK_3 = np.array(img_BRISK_2, dtype=np.uint8)
img_contr_copy = img_BRISK_3.copy()

plt.subplot(221)
plt.imshow(img_BRISK_3, cmap='gray')
kpts_1, des_1 = brisk.detectAndCompute(img_BRISK_3, None)

image_BRISK_1 = cv2.drawKeypoints(image=img_BRISK_3, outImage=img_BRISK_3, keypoints=kpts_1, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
plt.subplot(222)
plt.imshow(image_BRISK_1, cmap='gray')


# with enhanced contrast

inp_contr = ImageEnhance.Contrast(Image.fromarray(img_contr_copy))
contrast = 1.5
img_contr = inp_contr.enhance(contrast)
plt.subplot(223)
plt.imshow(img_contr, cmap='gray')
img_BRISK_4 = np.array(img_contr)
kpts_2, des_2 = brisk.detectAndCompute(img_BRISK_4, None)

image_BRISK_2 = cv2.drawKeypoints(image=img_BRISK_4, outImage=img_BRISK_4, keypoints=kpts_2, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
plt.subplot(224)
plt.imshow(image_BRISK_2, cmap='gray')

In [ ]:
X_train_images_np = np.expand_dims(np.array(X_train_images), axis = 3)
y_train_labels_np = np.array(Y_train_labels)
X_val_images_np = np.expand_dims(np.array(X_val_images), axis = 3)
y_val_labels_np = np.array(Y_val_labels)

print(X_train_images_np.shape)
print(y_train_labels_np.shape)
print(X_val_images_np.shape)
print(y_val_labels_np.shape)

print(X_train_images_np.dtype)
print(y_train_labels_np.dtype)
print(X_val_images_np.dtype)
print(y_val_labels_np.dtype)

In [ ]:
h5f2 = h5py.File('../input/luna-image-data/data-20211016T064224Z-001/data/testdataset.h5', 'r')
X_test_images = h5f2['X']
Y_test_labels = h5f2['Y']

X_test_images_np = np.expand_dims(np.array(X_test_images), axis = 3)
y_test_labels_np = np.array(Y_test_labels)

print(X_test_images_np.shape)
print(y_test_labels_np.shape)

#X_test_images_np.pad(0,78 ,78 ,0)
#print(y_test_images_np.shape)

In [ ]:
import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout,BatchNormalization,AveragePooling2D
from tensorflow.keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
import numpy as np 
import pandas as pd 
import os
import cv2
from tqdm import tqdm

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Input, merge, UpSampling2D, Cropping2D, ZeroPadding2D, Reshape, core, Convolution2D
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras import optimizers
from keras import backend as K
#from keras.optimizers import SGD
from keras.layers.merge import concatenate

from sklearn.metrics import fbeta_score
from sklearn.model_selection import train_test_split



In [ ]:
'''def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(50, 50, 1)))
	model.add(BatchNormalization())
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.1))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.2))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.3))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.4))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.4))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(BatchNormalization())
	model.add(Dropout(0.4))
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'));return model

mobile = define_model()'''

In [ ]:
#mobile.summary()

In [ ]:
'''op_layer = mobile.output
final_layer = Dense(128,activation='relu',kernel_initializer='he_uniform')(op_layer)
final_layer = Dense(128,activation='relu',kernel_initializer='he_uniform')(final_layer)
final_layer1 = Dense(128,activation='relu',kernel_initializer='he_uniform')(final_layer)
final_layer2 = Dense(2,activation= 'softmax')(final_layer1)
from keras.models  import Model
# Define model input and output
model1 = Model(inputs = mobile.input , outputs = final_layer2)
opt = SGD(lr=0.00029 , momentum=0.9)
import keras
optimizer_sgd = tensorflow.keras.optimizers.Adam(learning_rate=0.0007)
model1.compile(optimizer=optimizer_sgd, loss='categorical_crossentropy', metrics=['accuracy'])'''

In [ ]:
#model.summary()

In [ ]:
###########ALEX NET######################

In [ ]:
'''from tensorflow.keras import datasets, layers, models, losses
import tensorflow as tf
model = models.Sequential()
#model.add(layers.experimental.preprocessing.Resizing(50,50, interpolation="bilinear", input_shape=X_train_images_np.shape[1:]))
model.add(layers.Conv2D(32,(3,3), strides=1, padding='same',input_shape=(50, 50, 1)))
model.add(layers.Lambda(tf.nn.local_response_normalization))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.1))
model.add(layers.MaxPooling2D(3, strides=1))
model.add(layers.Conv2D(32,(3,3), strides=1, padding='same'))
#model.add(layers.Lambda(tf.nn.local_response_normalization))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.1))
model.add(layers.MaxPooling2D(3, strides=1))
model.add(layers.Conv2D(64, (3,3), strides=1, padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.1))
model.add(layers.MaxPooling2D(3, strides=1))
model.add(layers.Conv2D(64, (3,3), strides=1, padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.1))
model.add(layers.MaxPooling2D(3, strides=1))
model.add(layers.Conv2D(64, (3,3), strides=1, padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(3, strides=2))
model.add(layers.Conv2D(64, (3,3), strides=1, padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(3, strides=2))
model.add(layers.Dropout(0.1))
#model.add(layers.MaxPooling2D(3, strides=2))
model.add(layers.Conv2D(128, (3,3), strides=1, padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(3, strides=2))
model.add(layers.Dropout(0.2))
#model.add(layers.Lambda(tf.nn.local_response_normalization))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.2))
#model.add(layers.Dense(10, activation='softmax'))
model.summary()'''


In [ ]:
'''op_layer = model.output
final_layer = Dense(128,activation='relu',kernel_initializer='he_uniform')(op_layer)
#model.add(layers.Dropout(0.5))
#final_layer = Dense(128,activation='relu',kernel_initializer='he_uniform')(final_layer)
final_layer1 = Dense(128,activation='relu',kernel_initializer='he_uniform')(final_layer)
final_layer2 = Dense(2,activation= 'softmax')(final_layer1)
from keras.models  import Model
# Define model input and output
model = Model(inputs = model.input , outputs = final_layer2)
opt = SGD(lr=0.00029 , momentum=0.9)
import keras
optimizer_adam = tensorflow.keras.optimizers.Adam(learning_rate=0.0005)
model.compile(optimizer= optimizer_adam, loss='categorical_crossentropy', metrics=['accuracy'])'''

In [ ]:
##############################################################################################

In [ ]:
from keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from keras.callbacks import Callback
from tensorflow.keras.optimizers import Adam
from keras import backend as K
#from config import *
#from generators import get_seg_batch
from skimage import morphology, measure, segmentation
#from visual_utils import plot_slices, plot_comparison

In [ ]:
#model3 = unet()

In [ ]:
import matplotlib.pyplot as plt
def summarize_diagnostics(hist):
	plt.plot(hist.history["accuracy"])
	plt.plot(hist.history['val_accuracy'])
	plt.plot(hist.history['loss'])
	plt.plot(hist.history['val_loss'])
	plt.title("model accuracy")
	plt.ylabel("Accuracy")
	plt.xlabel("Epoch")
	plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint('classification_deep_conv_model.h5', save_best_only=True),
    keras.callbacks.ReduceLROnPlateau()]

In [ ]:
# run the test harness for evaluating a model
def run_test_harness():
	# create data generator
	datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True,zoom_range=0.1)
	# prepare iterator
	it_train = datagen.flow(X_train_images_np, y_train_labels_np, batch_size=64)
	# fit model
	steps = int(X_train_images_np.shape[0] / 64)
	hist = model3.fit_generator(it_train, steps_per_epoch=steps, epochs= 300, validation_data=(X_val_images_np, y_val_labels_np), verbose=1, callbacks = callbacks)
	# evaluate model ** change the epoches to 500
	_, acc = model3.evaluate(X_val_images_np, y_val_labels_np, verbose=1)
	print('> %.3f' % (acc * 100.0))
	# learning curves
	summarize_diagnostics(hist)

# entry point, run the test harness
run_test_harness()

In [ ]:
model_new_1 = keras.models.load_model("classification_deep_conv_model.h5")

model_new_1.summary()


In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
y_pred = model3.predict(X_test_images_np)
y_p = np.argmax(y_pred,axis=1)
y_true = np.argmax(y_test_labels_np,axis=1)
print('confusion matrix')
print(confusion_matrix(y_true,y_p))
cm = confusion_matrix(y_true,y_p)

In [ ]:
import seaborn as sn
plt.figure(figsize = (4, 4))
sn.heatmap(cm , annot = True)
plt.xlabel('True' , fontsize = 20)
plt.ylabel('Predicted' , fontsize = 20)


In [ ]:
print('Classification report')
print(classification_report(y_true,y_p))

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
op_layer = model_new_1.get_layer('dense_37').output
model_new = Model(inputs = model_new_1.input,outputs = op_layer)
train_new = sc.fit_transform(model_new.predict(X_train_images_np))
val_new = sc.fit_transform(model_new.predict(X_val_images_np))
test_new = sc.fit_transform(model_new.predict(X_test_images_np))


In [ ]:
from sklearn.svm import SVC
svm = SVC(kernel='rbf')
svm.fit(train_new,np.argmax(y_train_labels_np,axis=1))
print('Training score of svm',svm.score(train_new,np.argmax(y_train_labels_np,axis=1)))
print('validation score of svm :',svm.score(val_new,np.argmax(y_val_labels_np,axis=1)))
print('testing score of svm :',svm.score(test_new,np.argmax(y_test_labels_np,axis=1)))


In [ ]:
from xgboost import XGBClassifier
xg = XGBClassifier()
xg.fit(train_new,np.argmax(y_train_labels_np,axis=1))
print('Training score of xgb',xg.score(train_new,np.argmax(y_train_labels_np,axis=1)))
print('validation score of xgb :',xg.score(val_new,np.argmax(y_val_labels_np,axis=1)))
print('testing score of xgb :',xg.score(test_new,np.argmax(y_test_labels_np,axis=1)))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf.fit(train_new,np.argmax(y_train_labels_np,axis=1))
print('Training score of random_forest',rf.score(train_new,np.argmax(y_train_labels_np,axis=1)))
print('validation score of random_forest :',rf.score(val_new,np.argmax(y_val_labels_np,axis=1)))
print('testing score of random_forest :',rf.score(test_new,np.argmax(y_test_labels_np,axis=1)))

https://github.com/dv-123/Lung_cancer/blob/master/Documnets/final_report_lungcancer.docx

def define_model():
	model = Sequential()
	model.add(Conv2D(64, 3, 3, activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(50, 50, 1)))
  model.add(Dropout(0.2))
	#model.add(BatchNormalization())
	model.add(Conv2D(64, 3, 3 , activation='relu', kernel_initializer='he_uniform', padding='same'))
	#model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
 
	model.add(Conv2D(128, 3, 3 , activation='relu', kernel_initializer='he_uniform', padding='same'))
	#model.add(BatchNormalization())
	model.add(Dropout(0.2))
	model.add(Conv2D(128, 3, 3, activation='relu', kernel_initializer='he_uniform', padding='same'))
	 #model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
 
	model.add(Conv2D(256, 3, 3, activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(Dropout(0.2))
  #model.add(BatchNormalization())
	model.add(Conv2D(256, 3, 3, activation='relu', kernel_initializer='he_uniform', padding='same'))
	#model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
 
	model.add(Conv2D(512, 3, 3, activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(Dropout(0.2))
	#model.add(BatchNormalization())
	model.add(Conv2D(512, 3, 3, activation='relu', kernel_initializer='he_uniform', padding='same'))
	#model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	
	model.add(Conv2D(1024, 3, 3, activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(Dropout(0.2))
	#model.add(BatchNormalization())
	model.add(Conv2D(1024, 3, 3, activation='relu', kernel_initializer='he_uniform', padding='same'))
	#model.add(BatchNormalization())
	#model.add(MaxPooling2D((2, 2)))
	
	model.add(Dropout(0.5))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(BatchNormalization())
	model.add(Dropout(0.5))
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'));return model

mobile = define_model()

##########################################################################################

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
def down_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    p = keras.layers.MaxPool2D((2, 2), (2, 2))(c)
    return c, p

def up_block(x, skip, filters, kernel_size=(3, 3), padding="same", strides=1):
    us = keras.layers.UpSampling2D((2, 2))(x)
    concat = keras.layers.Concatenate()([us, skip])
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(concat)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

def bottleneck(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

In [ ]:
def UNet():
    f = [8, 16 , 32 , 64, 128 ]
    inputs = keras.layers.Input((50 ,50 ,  1))
    
    p0 = inputs
    c1, p1 = down_block(p0, f[0]) #128 -> 64
    c2, p2 = down_block(p1, f[1]) #64 -> 32
    c3, p3 = down_block(p2, f[2]) #32 -> 16
    c4, p4 = down_block(p3, f[3]) #16->8
    
    bn = bottleneck(p4, f[4])
    
    u1 = up_block(bn, c4, f[3]) #8 -> 16
    u2 = up_block(u1, c3, f[2]) #16 -> 32
    u3 = up_block(u2, c2, f[1]) #32 -> 64
    u4 = up_block(u3, c1, f[0]) #64 -> 128
    
    outputs = keras.layers.Conv2D(1, (1, 1), padding="same", activation="softmax")(u4)
    model = keras.models.Model(inputs, outputs)
    return model
mobile_1 = UNet()


In [ ]:
mobile_1.summary()